In [3]:
!pip install rdflib networkx scikit-learn matplotlib
!pip install rdflib
!pip list | grep python-louvain

!pip install python-louvain

'grep' n’est pas reconnu en tant que commande interne
ou externe, un programme exécutable ou un fichier de commandes.


In [5]:
from rdflib import URIRef, BNode, Literal
from rdflib import Namespace
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL
from rdflib import Graph, URIRef, Literal, BNode
from rdflib.namespace import FOAF, RDF
from rdflib import Namespace, Graph, URIRef
from rdflib.namespace import RDF
import networkx as nx
import matplotlib.pyplot as plt
import re
from rdflib import Graph, RDF, URIRef, Namespace


In [17]:

# Load the RDF data
akg_file = "teaching_akg.ttl"
g = Graph()
g.parse(akg_file, format="ttl")
akg_namespace = Namespace("http://sonfack.com/2023/12/tao/")
cao_namespace = Namespace("http://sonfack.com/2023/12/cao/")


# Define namespace and type for entities (adjust as needed)
TAO = Namespace("http://sonfack.com/2024/01/tao#")  # Replace with your actual namespace

# Extract and print relevant assertions
relevant_assertions = []

for subj, pred, obj in graph:
    # Exclude subjects and predicates starting with "has" or "is"
    if not str(pred).startswith("http://sonfack.com/2024/01/tao#has") and not str(pred).startswith("http://sonfack.com/2024/01/tao#is"):
        relevant_assertions.append((subj, pred, obj))

# Print relevant assertions
for subj, pred, obj in relevant_assertions:
    print(f"Subject: {subj.split('#')[-1]}\nPredicate: {pred.split('#')[-1]}\nObject: {obj.split('#')[-1]}\n" )

def read_all_activities(akg: Graph, as_str=True) -> list:
    """This function returns all activities of an activity knowledge graph
    - akg: an activity knowledge graph as parsed by RDFLib
    - as_str: (boolean) tells if the activities are simple str default = True
    """
    activities_list = [str(activity) if as_str else activity for activity in akg.subjects(predicate=RDF.type, object=cao_namespace.Activity, unique=True)]
    return activities_list




def read_akg_node(node_uri: str, akg:Graph, as_str=True) -> dict:
    """This function returns all elements directly linked to a akg node
    - activity_uri (string): the given activity uri in graph akg
    - akg (Graph): an activity knowledge graph as parsed by RDFLib
    """
    activity_info = {}
    activity_uri_ref = f"{akg_namespace}{node_uri}"
    print(activity_uri_ref)
    for act_predicate, act_object in akg.predicate_objects(subject=URIRef(activity_uri_ref)):
        pred = act_predicate
        obj = act_object
        if as_str:
            pred = str(act_predicate)
            obj = str(act_object)
        if pred in activity_info:
            existing_objects = activity_info[pred] + [obj]
            activity_info[pred] = existing_objects
        else:
            activity_info[pred] = [obj]
    return activity_info

print(read_all_activities(g, False))


def add_activity_to_nxgraph(G, activity_uri, activity_info):
    #print("ajout de l'activité", activity_uri)
    #print("affichage des informations", activity_info)
    activity_g = nx.DiGraph()

    for pred, obj_list in activity_info.items():
        pred_label = pred.split('/')[-1]
        if '#' in pred_label:
            pred_label = pred_label.split('#')[1]

        for obj in obj_list:
            obj_label = obj.split('/')[-1]
            node_info = read_akg_node(obj_label, g)

            if node_info:
                # Assurez-vous que vous récupérez une chaîne et non une liste
                obj_label = node_info.get('http://sonfack.com/2023/12/tao/hasName', [None])[0]
                if obj_label is None:
                    continue  # Passer à l'itération suivante si obj_label est None


            G.add_node(activity_uri)
            G.add_node(obj_label)
            G.add_edge(activity_uri, obj_label, label=pred_label)
            activity_g.add_node(activity_uri)
            activity_g.add_node(obj_label)
            activity_g.add_edge(activity_uri, obj_label, label=pred_label)

    visualize_activity(activity_g)


def visualize_activity(G):
  # Visualize the graph
  plt.figure(figsize=(12, 12))

  # Generate positions for nodes using a layout algorithm
  pos = nx.spring_layout(G, seed=42)

  # Draw the nodes and edges
  nx.draw(G, pos, with_labels=True, node_size=300, node_color="blue", font_size=5, font_weight="bold", edge_color="black")

  # Draw edge labels
  edge_labels = nx.get_edge_attributes(G, 'label')
  nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='green', font_size=8)

  # Show plot
  plt.title("Graphe de Connaissances des Activités")
  plt.show()

liste_activity = read_all_activities(g, as_str=True)
print(liste_activity)
for activity in liste_activity:
  subj = activity.split('/')[-1]
  info_activity = read_akg_node(subj, g, as_str=True)
  print(info_activity)
  add_activity_to_nxgraph(G,activity,info_activity)


# Visualize the graph
plt.figure(figsize=(12, 12))

# Generate positions for nodes using a layout algorithm
pos = nx.spring_layout(G, seed=42)

# Draw the nodes and edges
nx.draw(G, pos, with_labels=True, node_size=300, node_color="blue", font_size=5, font_weight="bold", edge_color="black")

# Draw edge labels
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red', font_size=8)

# Show plot
plt.title("Graphe de Connaissances des Activités")
plt.show()



# Liste de vos informations RDF (exemple)
data = [
    {"subject": "http://sonfack.com/2023/12/tao/book-9d0de03c-dcc8-4a6c-8dd3-3b921cba2d1b", "predicate": "type", "object": "http://sonfack.com/2023/12/tao/HardDocument"},
    {"subject": "http://sonfack.com/2023/12/tao/Moral", "predicate": "subClassOf", "object": "http://sonfack.com/2023/12/cao/Subject"},
    # ... ajoutez d'autres données ici
]


# Initialize NetworkX graph
G = nx.DiGraph()  # Using directed graph to capture the direction of relationships

# Extract and add relevant assertions to the graph
for subj, pred, obj in graph:
    if not str(pred).startswith("http://sonfack.com/2024/01/tao#has#") and not str(pred).startswith("http://sonfack.com/2024/01/tao#is#"):
        subj_label = subj.split('#')[-1]
        pred_label = pred.split('#')[-1]
        obj_label = obj.split('#')[-1]
        G.add_node(subj_label)  # Add subject node
        G.add_node(obj_label)   # Add object node
        G.add_edge(subj_label, obj_label, label=pred_label)  # Add edge with predicate as label

        # Print the extracted information
        # print(f"Subject: {subj_label}\nPredicate: {pred_label}\nObject: {obj_label}\n")


        # Fonction pour extraire la partie du nom après le namespace et retirer les chiffres
def extract_name_without_numbers(uri, namespace):
    match = re.match(f"{re.escape(namespace)}(.+)", uri)
    if match:
        name_with_numbers = match.group(1)
        # Retirer les chiffres
        name_without_numbers = re.sub(r'\d+', '', name_with_numbers)
        return name_without_numbers.strip('-')  # Retirer les tirets restants
    return uri

# Namespace à utiliser pour extraire les noms
namespace = "http://sonfack.com/2023/12/tao/"

# Traitement des données
for item in data:
    subject_name = extract_name_without_numbers(item['subject'], namespace)
    predicate = item['predicate']
    object_name = extract_name_without_numbers(item['object'], namespace)
    print(f"Subject: {subject_name}\nPredicate: {predicate}\nObject: {object_name}\n")


      # Positionnement des nœuds pour la visualisation
pos = nx.spring_layout(G)

# Visualize the graph
plt.figure(figsize=(12, 12))

# Generate positions for nodes using a layout algorithm
pos = nx.spring_layout(G, seed=42)

# Draw the nodes and edges
nx.draw(G, pos, with_labels=True, node_size=300, node_color="blue", font_size=5, font_weight="bold", edge_color="black")

# Draw edge labels
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red', font_size=10)

# Show plot
plt.title("Graphe de Connaissances des Activités")
plt.show()


# Convertir le graphe en non dirigé
G_undirected = G.to_undirected()

# Appliquer l'algorithme de Louvain
partitions = community_louvain.best_partition(G_undirected)

# Afficher le nombre de communautés détectées
print(f"Nombre de communautés détectées: {len(set(partitions.values()))}")



# Visualisation des communautés
# Exemples d'activité et d'informations à ajouter (à ajuster selon vos données)
activity_uri = "http://example.com/activity1"
activity_info = {
    "http://example.com/predicate1": ["http://example.com/object1", "http://example.com/object2"],
    "http://example.com/predicate2": ["http://example.com/object3"]
}

# Appeler la fonction pour ajouter l'activité au graphe
add_activity_to_nxgraph(G, activity_uri, activity_info)
plt.figure(figsize=(22, 22))
pos = nx.spring_layout(G_undirected, seed=42)
cmap = plt.get_cmap('viridis')
nx.draw_networkx_nodes(G_undirected, pos, node_size=300, node_color=list(partitions.values()), cmap=cmap, alpha=1)
nx.draw_networkx_edges(G_undirected, pos, edge_color="black", alpha=1)
nx.draw_networkx_labels(G_undirected, pos, font_size=8)
plt.title("Détection de Communautés")
plt.show()



def read_all_activities(akg: Graph, as_str=True) -> list:
    """Retourne toutes les activités d'un graphe de connaissances."""
    activities_list = [str(activity) if as_str else activity for activity in akg.subjects(predicate=RDF.type, object=cao_namespace.Activity, unique=True)]
    return activities_list

def read_akg_node(node_uri: str, akg: Graph, as_str=True) -> dict:
    """Retourne tous les éléments directement liés à un nœud du graphe."""
    activity_info = {}
    activity_uri_ref = f"{akg_namespace}{node_uri}"
    for act_predicate, act_object in akg.predicate_objects(subject=URIRef(activity_uri_ref)):
        pred = act_predicate
        obj = act_object
        if as_str:
            pred = str(act_predicate)
            obj = str(act_object)
        if pred in activity_info:
            existing_objects = activity_info[pred] + [obj]
            activity_info[pred] = existing_objects
        else:
            activity_info[pred] = [obj]
    return activity_info

def add_activity_to_nxgraph(G, activity_uri, activity_info):
    """Ajoute les activités et leurs relations au graphe NetworkX."""
    for pred, obj_list in activity_info.items():
        pred_label = pred.split('/')[-1]
        if '#' in pred_label:
            pred_label = pred_label.split('#')[1]

        for obj in obj_list:
            obj_label = obj.split('/')[-1]
            node_info = read_akg_node(obj_label, graph)

            if node_info:
                # Assurez-vous que vous récupérez une chaîne et non une liste
                obj_label = node_info.get('http://sonfack.com/2023/12/tao/hasName', [None])[0]
                if obj_label is None:
                    continue

            G.add_node(activity_uri)
            G.add_node(obj_label)
            G.add_edge(activity_uri, obj_label, label=pred_label)

def visualize_activity(G):
    """Visualiser le graphe de connaissances."""
    plt.figure(figsize=(12, 12))
    pos = nx.spring_layout(G, seed=42)
    nx.draw(G, pos, with_labels=True, node_size=300, node_color="blue", font_size=5, font_weight="bold", edge_color="black")
    edge_labels = nx.get_edge_attributes(G, 'label')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='green', font_size=8)
    plt.title("Graphe de Connaissances des Activités")
    plt.show()

# Charger le fichier RDF dans un graphe NetworkX
G = nx.DiGraph()

# Lire toutes les activités
liste_activity = read_all_activities(graph, as_str=True)

# Ajouter chaque activité au graphe
for activity in liste_activity:
    subj = activity.split('/')[-1]
    info_activity = read_akg_node(subj, graph, as_str=True)
    add_activity_to_nxgraph(G, activity, info_activity)

# Détection des communautés
partition = community_louvain.best_partition(G.to_undirected())

# Visualiser les communautés
plt.figure(figsize=(12, 12))
pos = nx.spring_layout(G, seed=42)
colors = [partition[node] for node in G.nodes()]
nx.draw(G, pos, node_color=colors, with_labels=True, node_size=300, cmap=plt.cm.jet, font_size=5, font_weight="bold", edge_color="black")
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red', font_size=10)
plt.title("Communautés dans le Graphe de Connaissances des Activités")
plt.show()





FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\teaching_akg.ttl'